In [1]:
import numpy
import netCDF4
from osgeo import gdal, osr
from pyproj import Proj, transform
import ogr
import os
import pandas as pd
import datetime

#catchmentRasterFile = 'C:/DDrive/Beckers/construction/Beckers DEM Raster.asc'
#catchmentRasterFile = 'C:/DDrive/Beckers/construction/becks_subcats.asc'
#catchmentRasterFile = 'P:/projects/SILO/Wes/MNDBSpatial/MNDB Catchments Raster.asc'
#catchmentCategoriesFile = 'P:/projects/SILO/Wes/MNDBSpatial/MNDB Catchments Raster_categories.csv'
#catchmentRasterFile = 'P:/projects/SILO/Wes/CBSpatial/CB Catchments Raster.asc'
#catchmentCategoriesFile = 'P:/projects/SILO/Wes/CBSpatial/CB Catchments Raster_categories.csv'
catchmentRasterFile = 'P:/projects/SILO/Wes/SWNRMSpatial/SWNRM Catchments Raster.asc'
catchmentCategoriesFile = 'P:/projects/SILO/Wes/SWNRMSpatial/SWNRM Catchments Raster_categories.csv'

outGeoCoordsFile = 'P:/projects/SILO/Wes/SWNRMGeoCoordsPython.csv'

catFileRasID = 'Ras_ID'
catFileCatName = 'Ras_Cat'
siloDataFile = 'P:/projects/SILO/2000.daily_rain.nc'

#Col names for GeoCoords CSV
catGrpName = 'CATCHGROUP'
latName = 'SILO_LAT'
longName = 'SILO_LONG'
countName = 'COUNT'



In [2]:
#Make dictionary of catchment raster values
catchRasVals = {}

catchRasDF = pd.read_csv(catchmentCategoriesFile)
for index, row in catchRasDF.iterrows():
    if not row[catFileRasID] in catchRasVals:
        #print('Adfing: ' + str(row[catFileRasID]) + ' and ' + str(row[catFileCatName]))
        catchRasVals[row[catFileRasID]] = row[catFileCatName]

print(len(catchRasVals))

96


In [3]:
theCatchDataSet = gdal.Open(catchmentRasterFile)
catchGT = theCatchDataSet.GetGeoTransform()
print(catchGT)

(957009.188100002, 30.0, 0.0, -2736867.3393, 0.0, -30.0)


In [4]:
gdalSilo = gdal.Open(siloDataFile)
siloGT = gdalSilo.GetGeoTransform()
print(siloGT)

(0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


In [5]:
print( "Driver: ",gdalSilo.GetDriver().ShortName, gdalSilo.GetDriver().LongName)
print( "Size is ", gdalSilo.RasterXSize, gdalSilo.RasterYSize)
print( "Bands = ", gdalSilo.RasterCount)
print( "Coordinate System is:", gdalSilo.GetProjectionRef ())
print( "GetGeoTransform() = ", gdalSilo.GetGeoTransform ())

Driver:  HDF5Image HDF5 Dataset
Size is  841 681
Bands =  366
Coordinate System is: 
GetGeoTransform() =  (0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


In [6]:
print("Starting matchy match at: " + str(datetime.datetime.now()))
##Need this to calc pixel size as well as work out order of y-axis
netCDFData = netCDF4.Dataset(siloDataFile, 'r')

#Grab the info we need to build GeoTransform object
#Checking for order of y-axis coordinates, would be needed if writing raster form Numpy array

lats = netCDFData.variables['lat']
lons = netCDFData.variables['lon']
times = netCDFData.variables['time']

first_lat = lats[:1]
last_lat = lats[len(lats)-1:]
#If first_lat is LESS THAN last_lat, numpy array would be written to raster upside down...

first_lon = lons[:1]
last_lon = lons[len(lons)-1:]

colCount = gdalSilo.RasterXSize
rowCount = gdalSilo.RasterYSize

pixWidth = (last_lon - first_lon) / (gdalSilo.RasterXSize - 1)
pixHeight = (last_lat - first_lat) / (gdalSilo.RasterYSize - 1)

if first_lat < last_lat:
    #adjust for our use
    pixHeight *= -1


siloLeft = min(lons) - (0.5 * pixWidth)#adjusting to get cell edge, not centre
siloTop = max(lats) - (0.5 * pixHeight)#adjusting to get cell edge, not centre

netCDFData.close()

catchProjSRS = theCatchDataSet.GetProjection()
catchProjConverter = osr.SpatialReference()
catchProjConverter.ImportFromWkt(catchProjSRS)  # populates the spatial ref object with our WKT SRS
catchProj_forPyProj = catchProjConverter.ExportToProj4()  # Exports an SRS ref as a Proj4 string usable by PyProj

#print(catchProj_forPyProj)

wgsGeo = osr.SpatialReference()
wgsGeo.ImportFromEPSG(4326)#WGS84
wgsGeo_forPyProj = wgsGeo.ExportToProj4()
#print(wgsGeo_forPyProj)

coordTransform = osr.CoordinateTransformation(catchProjConverter, wgsGeo)

#print(siloLeft)
#print(siloTop)
#print(pixWidth)
#print(pixHeight)


#create an empty raster in memory of requisite GeoTransform & Projection
tempDS = gdal.GetDriverByName("MEM").Create('', colCount, rowCount, 1, gdal.GDT_Int32)#GTiff
tempDS.SetGeoTransform((siloLeft, pixWidth, 0, siloTop, 0, pixHeight))

siloGT = tempDS.GetGeoTransform()

statsDict = {}

catchBand = theCatchDataSet.GetRasterBand(1)
catchNumpy = catchBand.ReadAsArray()
catND = catchBand.GetNoDataValue()

#RasterYSize is number of rows
#RasterXSize is number of cols
rowCount = theCatchDataSet.RasterYSize
colCount = theCatchDataSet.RasterXSize
for rowid in range(0, rowCount):#theCatchDataSet.RasterYSize
    for colid in range(0, colCount):#theCatchDataSet.RasterXSize
        
        catchVal = catchNumpy[rowid, colid]
        #print('Pixelval: ' + str(catchVal) + ' col: ' + str(rowid) + ' row: ' + str(colid))
        
        if catchVal == catND:
            continue
        
        #Could put this into a function
        #xp = catchGT[0] * colid + catchGT[1] * rowid + xoff
        centX = catchGT[0] + (catchGT[1] * colid) + (catchGT[1] / 2)#Need the  + (catchGT[1] / 2) to get to centre of pixel
        centY = catchGT[3] + (catchGT[5] * rowid) + (catchGT[5] / 2)#Need the   + (catchGT[5] / 2 to get to centre of pixel
        
        # create a geometry from coordinates
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(centX, centY)
        
        # transform point
        point.Transform(coordTransform)
        
        #this conversion to int actually seems to work for silo stuff...
        #149.07453353146647 ultimately ends up being assigned to Silo -149.05
        #149.08068461867092 ends up being assigned to Silo -149.10
        geoXcell = int((point.GetX() - siloGT[0]) / siloGT[1]) #x pixel
        geoYcell = int((point.GetY() - siloGT[3]) / siloGT[5]) #y pixel
        
        centSiloX = siloGT[0] + (siloGT[1] * geoXcell) + (siloGT[1] / 2)
        centSiloY = siloGT[3] + (siloGT[5] * geoYcell) + (siloGT[5] / 2)
        
        #print('Pixelval: ' + str(catchVal) + ' Lat: ' + str(point.GetX()) + ' Long: ' + str(point.GetY())
        #      + ' siloX: ' + str(geoXcell) + ' siloY: ' + str(geoYcell)
        #      + ' siloLong: ' + str(centSiloX) + ' siloLat: ' + str(centSiloY))
        
        #Can't use a geometry object as a simple key in a dictionary
        #siloPoint = ogr.Geometry(ogr.wkbPoint)
        #siloPoint.AddPoint(centSiloX, centSiloY)
        siloPointKey = str(round(centSiloX,2)) + ',' + str(round(centSiloY,2))
        

        if not catchVal in statsDict:
            statsDict[catchVal] = {}
        
        if not siloPointKey in statsDict[catchVal]:
            statsDict[catchVal][siloPointKey] = 0
        
        statsDict[catchVal][siloPointKey] += 1


print("All Done matching at: " + str(datetime.datetime.now()))
#print(statsDict)




Starting matchy match at: 2019-07-17 15:02:13.467520
All Done matching at: 2019-07-17 15:48:55.354750


In [7]:
#Now convert to Geoocords CSV file
#via dataframe
geoCoordsDF = pd.DataFrame(columns=[catGrpName,latName,longName,countName])

for catNum, coordGenerals in statsDict.items():
    
    catName = catchRasVals[catNum]
    #print(catName)
    
    for coordPoint, coordCount in coordGenerals.items():
        
        pointDets = coordPoint.split(',')
        
        geoCoordsDF.loc[len(geoCoordsDF)]=[catName, pointDets[1], pointDets[0], str(coordCount)]
        #geoCoordsDF.append([catName, pointDets[1], pointDets[0]], str(coordCount))
    

geoCoordsDF.to_csv(outGeoCoordsFile, index=False)

print('Finished at ' + str(datetime.datetime.now()))
    

Finished at 2019-07-17 15:49:05.357775
